In [2]:
import pandas as pd
import gzip
import os
from geopy.geocoders import Nominatim
from geopy.point import Point
import geopandas as gpd
from shapely.geometry import Point

---

<h1><center>Bussiness Google</center></h1>

<h1><center>Lectura de información</center></h1>

El primer paso antes de realizar la normalización y limpieza de los datasets correspondientes a los metadatos de google, fue unificar los ya mencionados datasets para luego proceder a realizar la exportación de los mismos.

In [4]:
# En este bloque, realize la apertura de todos los .json de meta datos de google, para luego concatenarlos y formar un solo dataframe con toda la informacion

dicc = {}

for x in range(1,12):
    dicc[x] = pd.read_json(f'..\datasets\extras\{x}.json', lines=True)
    
# Lista para almacenar los DataFrames
lista_dataframes = []

# Iterar sobre los valores del diccionario (los Datasets<.json>)
for df in dicc.values():
    lista_dataframes.append(df)

# Concatenar los DataFrames en uno solo
resultado_concatenado = pd.concat(lista_dataframes, ignore_index=True)



### Cabe aclarar que este bloque de comandos solo se utiliza en primera instancia, ya que luego se trabajara sobre la exportación unificada

In [5]:
resultado_concatenado.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [6]:
resultado_concatenado.to_parquet(r'..\datasets\extras\resultado_google.parquet.gz',index = False,compression='gzip')

In [13]:
# Lectura de dataset unificado  
meta_datos_google = pd.read_parquet(r'..\datasets\extras\resultado_google.parquet.gz')

meta_datos_google.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [14]:
print(resultado_concatenado.columns.values)

['name' 'address' 'gmap_id' 'description' 'latitude' 'longitude'
 'category' 'avg_rating' 'num_of_reviews' 'price' 'hours' 'MISC' 'state'
 'relative_results' 'url']


## Columnas eliminadas

-state (porque no es relevante, dice si esta abierto el local o no)

-url (se puede ingresar utilizando la siguiente url mas el gmap-id) [https://www.google.com/maps/place//data=!4m2!3m1!1s {aca va el gmap_id}]

-description (esta información podemos obtenerla en caso de ser necesario usando la api, solo tiene 250.000 datos lo que vuelve a la columna irrelevante en comparación con la cantidad total, por eso mismo se recomienda realizar la extracción de dicha columna de ser necesario desde la api)

-MISC (esta columna tiene una descripcion de como estan las condiciones del local)

-price (es irrelevante el precio)

-num_of_reviews (esta columna tendrémos que volver a crearla luego de realizar el ETL de los estados)

-address (esta información podemos extraerla del url en el caso de ser necesario)

-hours (en caso de ser necesario se extraeran los datos específicos de la url)

-relative_result (podemos obtener el mismo resultado y hasta mas eficiente realizando una ponderación de los valores de latitude y longitude)

Muestra de acceso a google maps por medio de url mas gmap_id:

https://www.google.com/maps/place//data=!4m2!3m1!1s0x88e77e406e2da09d:0x479c876de5fd4ec4

In [15]:
meta_datos_google.drop(columns=['state','url','description','MISC','price','num_of_reviews','address','hours','relative_results'],inplace=True)

In [16]:
meta_datos_google

,name,gmap_id,latitude,longitude,category,avg_rating
0,Porter Pharmacy,0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,[Pharmacy],4.9
1,City Textile,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.215290,[Textile exporter],4.5
2,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,[Korean restaurant],4.4
3,Nova Fabrics,0x80c2c89923b27a41:0x32041559418d447,34.023669,-118.232930,[Fabric store],3.3
4,Nobel Textile Co,0x80c2c632f933b073:0xc31785961fe826a6,34.036694,-118.249421,[Fabric store],4.3
...,...,...,...,...,...,...
3025006,Steak 48,0x89c6c74f43a49b55:0x6be6995921c58b12,39.947254,-75.164953,"[Steak house, Fine dining restaurant, Seafood ...",4.6
3025007,"Jack Mcnerney Chevrolet, Inc.",0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,42.788636,-76.122120,[Chevrolet dealer],4.4
3025008,Central Ny Spay Neuter Assista,0x89da6b9ce832726f:0xbd0d3630f1e3b42e,42.603831,-76.165286,[Veterinarian],4.3
3025009,Ok Feed Store,0x88d9e872a9e39f3d:0x5de29f50507e5d73,25.491641,-80.456076,[Animal feed store],4.9


In [18]:
bussiness_google = meta_datos_google.copy()

In [19]:
#bussiness_google = pd.read_json(r'..\datasets\processed\google\bussiness_google.json.gz')
bussiness_google

,name,gmap_id,latitude,longitude,category,avg_rating
0,Porter Pharmacy,0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,[Pharmacy],4.9
1,City Textile,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.215290,[Textile exporter],4.5
2,San Soo Dang,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,[Korean restaurant],4.4
3,Nova Fabrics,0x80c2c89923b27a41:0x32041559418d447,34.023669,-118.232930,[Fabric store],3.3
4,Nobel Textile Co,0x80c2c632f933b073:0xc31785961fe826a6,34.036694,-118.249421,[Fabric store],4.3
...,...,...,...,...,...,...
3025006,Steak 48,0x89c6c74f43a49b55:0x6be6995921c58b12,39.947254,-75.164953,"[Steak house, Fine dining restaurant, Seafood ...",4.6
3025007,"Jack Mcnerney Chevrolet, Inc.",0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,42.788636,-76.122120,[Chevrolet dealer],4.4
3025008,Central Ny Spay Neuter Assista,0x89da6b9ce832726f:0xbd0d3630f1e3b42e,42.603831,-76.165286,[Veterinarian],4.3
3025009,Ok Feed Store,0x88d9e872a9e39f3d:0x5de29f50507e5d73,25.491641,-80.456076,[Animal feed store],4.9


In [20]:
illinois = gpd.read_file(r'..\datasets\raw\IL.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_google['longitude'], bussiness_google['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_google, geometry=geometry)
# Realizar la unión espacial
empresas_en_illinois_google = gpd.sjoin(meta_datos_google, illinois, op='within')

C:\Users\jonat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\jonat\AppData\Local\Temp\ipykernel_1920\2852414068.py:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  empresas_en_illinois_google = gpd.sjoin(meta_datos_google, illinois, op='within')


In [21]:
california = gpd.read_file(r'..\datasets\raw\CA.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_google['longitude'], bussiness_google['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_google, geometry=geometry)
# Realizar la unión espacial
empresas_en_california_google = gpd.sjoin(meta_datos_google, california, op='within')

C:\Users\jonat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\jonat\AppData\Local\Temp\ipykernel_1920\3222356173.py:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  empresas_en_california_google = gpd.sjoin(meta_datos_google, california, op='within')


In [22]:
florida = gpd.read_file(r'..\datasets\raw\FL.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_google['longitude'], bussiness_google['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_google, geometry=geometry)
# Realizar la unión espacial
empresas_en_florida_google = gpd.sjoin(meta_datos_google, florida, op='within')

C:\Users\jonat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\jonat\AppData\Local\Temp\ipykernel_1920\138000119.py:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  empresas_en_florida_google = gpd.sjoin(meta_datos_google, florida, op='within')


In [23]:
Nuevajersey = gpd.read_file(r'..\datasets\raw\NJ.geo.json')
# Crear un GeoDataFrame a partir del DataFrame existente
geometry = [Point(xy) for xy in zip(bussiness_google['longitude'], bussiness_google['latitude'])]
meta_datos_google = gpd.GeoDataFrame(bussiness_google, geometry=geometry)
# Realizar la unión espacial
empresas_en_Nuevajersey_google = gpd.sjoin(meta_datos_google, Nuevajersey, op='within')

C:\Users\jonat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\jonat\AppData\Local\Temp\ipykernel_1920\2282871996.py:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  empresas_en_Nuevajersey_google = gpd.sjoin(meta_datos_google, Nuevajersey, op='within')


In [24]:
print(len(empresas_en_florida_google) + len(empresas_en_california_google) + len(empresas_en_illinois_google) + len(empresas_en_Nuevajersey_google))

703889


In [25]:
# Lista de DataFrames a concatenar
dataframes = [empresas_en_florida_google, empresas_en_california_google, empresas_en_illinois_google, empresas_en_Nuevajersey_google]

# Concatenar los DataFrames verticalmente
empresas_concatenadas_google = pd.concat(dataframes, ignore_index=True)

In [27]:
empresas_concatenadas_google

,name_left,gmap_id,latitude,longitude,category,avg_rating,geometry,index_right,id,fips,name_right
0,NTK OUTDOOR,0x88d9beb4fe0532c1:0xef0555c169299d6,25.795204,-80.366038,[Corporate office],5.0,POINT (-80.36604 25.79520),0,USA-FL,12,Florida
1,Cruises Inc. - Connie Stewart,0x88c2e49b79f06c31:0x4ed8c8ce27e926e0,27.832187,-82.704805,[Cruise agency],5.0,POINT (-82.70480 27.83219),0,USA-FL,12,Florida
2,Seminole Family Restaurant,0x88c2fb8614e57ce9:0x40b51628875ad6e9,27.834885,-82.787419,None,3.6,POINT (-82.78742 27.83488),0,USA-FL,12,Florida
3,First Impressions Barbershop Inc.,0x88e711bd2244fe3b:0x8406dd780f0574d1,28.877581,-81.281276,[Barber shop],4.0,POINT (-81.28128 28.87758),0,USA-FL,12,Florida
4,"Brian Shaheen, MD",0x8893863ea87bd5dd:0x9383ebf973e74abb,30.159982,-85.752277,"[Family practice physician, General practitioner]",4.2,POINT (-85.75228 30.15998),0,USA-FL,12,Florida
...,...,...,...,...,...,...,...,...,...,...,...
703884,AMTRAK DURANT YARD,0x89c252b98d7621fd:0x4ac46f3bd7756984,40.683651,-74.203041,[Railroad company],4.3,POINT (-74.20304 40.68365),0,USA-NJ,34,New Jersey
703885,T. R. Smiles Dental,0x89c1766b2682309d:0x4677876d0012f9a6,39.972538,-74.241226,"[Dental clinic, Dentist, General practitioner]",4.9,POINT (-74.24123 39.97254),0,USA-NJ,34,New Jersey
703886,Paisano's Barber Shop,0x89c17b4f55ec117b:0x87fd46b1a8175b52,40.121928,-74.393946,[Barber shop],4.7,POINT (-74.39395 40.12193),0,USA-NJ,34,New Jersey
703887,Jackson Pizza,0x89c17b3b65b4680d:0x40a43d960a97a0,40.109785,-74.387818,"[Pizza restaurant, Italian restaurant, Pizza d...",4.5,POINT (-74.38782 40.10979),0,USA-NJ,34,New Jersey


In [ ]:
# Lista de nombres de columnas a eliminar
columnas_a_eliminar = ['geometry', 'index_right', 'id', 'fips']

# Eliminar las columnas de empresas_concatenadas
empresas_concatenadas_google = empresas_concatenadas_google.drop(columns=columnas_a_eliminar)

empresas_concatenadas_google.rename(columns={'name_right': 'state'}, inplace=True)
empresas_concatenadas_google.rename(columns={'name_left': 'name'}, inplace=True)
empresas_concatenadas_google.rename(columns={'avg_rating': 'stars'}, inplace=True)

empresas_concatenadas_google

In [42]:
empresas_concatenadas_google

,name,gmap_id,latitude,longitude,category,stars,state
0,NTK OUTDOOR,0x88d9beb4fe0532c1:0xef0555c169299d6,25.795204,-80.366038,[Corporate office],5.0,Florida
1,Cruises Inc. - Connie Stewart,0x88c2e49b79f06c31:0x4ed8c8ce27e926e0,27.832187,-82.704805,[Cruise agency],5.0,Florida
2,Seminole Family Restaurant,0x88c2fb8614e57ce9:0x40b51628875ad6e9,27.834885,-82.787419,None,3.6,Florida
3,First Impressions Barbershop Inc.,0x88e711bd2244fe3b:0x8406dd780f0574d1,28.877581,-81.281276,[Barber shop],4.0,Florida
4,"Brian Shaheen, MD",0x8893863ea87bd5dd:0x9383ebf973e74abb,30.159982,-85.752277,"[Family practice physician, General practitioner]",4.2,Florida
...,...,...,...,...,...,...,...
703884,AMTRAK DURANT YARD,0x89c252b98d7621fd:0x4ac46f3bd7756984,40.683651,-74.203041,[Railroad company],4.3,New Jersey
703885,T. R. Smiles Dental,0x89c1766b2682309d:0x4677876d0012f9a6,39.972538,-74.241226,"[Dental clinic, Dentist, General practitioner]",4.9,New Jersey
703886,Paisano's Barber Shop,0x89c17b4f55ec117b:0x87fd46b1a8175b52,40.121928,-74.393946,[Barber shop],4.7,New Jersey
703887,Jackson Pizza,0x89c17b3b65b4680d:0x40a43d960a97a0,40.109785,-74.387818,"[Pizza restaurant, Italian restaurant, Pizza d...",4.5,New Jersey


In [ ]:
empresas_concatenadas_google.to_parquet(r'..\datasets\extras\empresas_concatenadas_filtradas_estados.parquet.gz',index = False, compression = 'gzip')

In [3]:
empresas_concatenadas_google = pd.read_parquet(r'..\datasets\extras\empresas_concatenadas_filtradas_estados.parquet.gz')
empresas_concatenadas_google

,name,gmap_id,latitude,longitude,category,stars,state
0,NTK OUTDOOR,0x88d9beb4fe0532c1:0xef0555c169299d6,25.795204,-80.366038,[Corporate office],5.0,Florida
1,Cruises Inc. - Connie Stewart,0x88c2e49b79f06c31:0x4ed8c8ce27e926e0,27.832187,-82.704805,[Cruise agency],5.0,Florida
2,Seminole Family Restaurant,0x88c2fb8614e57ce9:0x40b51628875ad6e9,27.834885,-82.787419,None,3.6,Florida
3,First Impressions Barbershop Inc.,0x88e711bd2244fe3b:0x8406dd780f0574d1,28.877581,-81.281276,[Barber shop],4.0,Florida
4,"Brian Shaheen, MD",0x8893863ea87bd5dd:0x9383ebf973e74abb,30.159982,-85.752277,"[Family practice physician, General practitioner]",4.2,Florida
...,...,...,...,...,...,...,...
703884,AMTRAK DURANT YARD,0x89c252b98d7621fd:0x4ac46f3bd7756984,40.683651,-74.203041,[Railroad company],4.3,New Jersey
703885,T. R. Smiles Dental,0x89c1766b2682309d:0x4677876d0012f9a6,39.972538,-74.241226,"[Dental clinic, Dentist, General practitioner]",4.9,New Jersey
703886,Paisano's Barber Shop,0x89c17b4f55ec117b:0x87fd46b1a8175b52,40.121928,-74.393946,[Barber shop],4.7,New Jersey
703887,Jackson Pizza,0x89c17b3b65b4680d:0x40a43d960a97a0,40.109785,-74.387818,"[Pizza restaurant, Italian restaurant, Pizza d...",4.5,New Jersey


In [5]:
categorias = []
# Itero por la columna categories del dataframe, para extraer todos sus valores y convertirlos en listas, los que luego agrego a la lista categorias
for x in empresas_concatenadas_google['category']:
    categorias.append(x)
categorias

[array(['Corporate office'], dtype=object),
 array(['Cruise agency'], dtype=object),
 None,
 array(['Barber shop'], dtype=object),
 array(['Family practice physician', 'General practitioner'], dtype=object),
 array(['Restaurant'], dtype=object),
 array(['Buffet restaurant'], dtype=object),
 None,
 array(['Mailing service', 'Courier service', 'Fax service',
        'Lamination service', 'Mailbox rental service', 'Notary public',
        'Packaging supply store', 'Post office', 'Shipping service'],
       dtype=object),
 array(['Pediatrician', "Children's hospital", 'Doctor'], dtype=object),
 array(['Tax preparation service', 'Accountant', 'Payroll service'],
       dtype=object),
 array(['Air conditioning system supplier', 'Air filter supplier',
        'Commercial refrigerator supplier', 'Distribution service',
        'Heating equipment supplier', 'Wholesaler'], dtype=object),
 array(['Hair salon'], dtype=object),
 array(['Locksmith'], dtype=object),
 array(['Laundromat'], dtype=objec

In [8]:
filtro_cat = []
for x in categorias:
    if x is None:
        pass
    else:
        for i in x:
            filtro_cat.append(i)

filtro_cat

['Corporate office',
 'Cruise agency',
 'Barber shop',
 'Family practice physician',
 'General practitioner',
 'Restaurant',
 'Buffet restaurant',
 'Mailing service',
 'Courier service',
 'Fax service',
 'Lamination service',
 'Mailbox rental service',
 'Notary public',
 'Packaging supply store',
 'Post office',
 'Shipping service',
 'Pediatrician',
 "Children's hospital",
 'Doctor',
 'Tax preparation service',
 'Accountant',
 'Payroll service',
 'Air conditioning system supplier',
 'Air filter supplier',
 'Commercial refrigerator supplier',
 'Distribution service',
 'Heating equipment supplier',
 'Wholesaler',
 'Hair salon',
 'Locksmith',
 'Laundromat',
 'Produce wholesaler',
 'Cooperative',
 'Physical therapy clinic',
 'Flooring store',
 'Bakery',
 'Deli',
 'Barber shop',
 'Boat builders',
 'Boat dealer',
 'Clothing store',
 'Real estate developer',
 'Logistics service',
 'Warehouse',
 'Real estate agency',
 'Hair salon',
 "Women's clothing store",
 'Boutique',
 'Clothing store',
 'D

In [25]:
restaurant_cat = []

# Itero sobre cada fila de la columna categories, con la finalidad de filtrar todos los comercios que corresponden a restaurantes
for x in filtro_cat:
    if 'estaur' in x:
        restaurant_cat.append(x)
        
# Con esta conversion elimino los valores repetidos de toda mi lista
restaurant_cat = set(restaurant_cat)

len(restaurant_cat)

263

In [32]:
empresas_concatenadas_google = empresas_concatenadas_google.dropna(subset=['category'])

In [33]:
nuevo_dataset = empresas_concatenadas_google[empresas_concatenadas_google['category'].apply(lambda x: any(item in restaurant_cat for item in x))]
nuevo_dataset

,name,gmap_id,latitude,longitude,category,stars,state
5,Cape Seafood Shack,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,26.641377,-81.940545,[Restaurant],5.0,Florida
6,Fresh Point Country Buffet,0x88c2e4e34f1ed783:0x76c5da381c499d79,27.867489,-82.702971,[Buffet restaurant],5.0,Florida
48,Hot Box,0x8890b9241e704667:0x3a1e565c17c00993,30.391411,-87.267220,[Restaurant],4.2,Florida
83,Shell,0x88e635378f43352f:0xa1b53c63436fa428,29.183272,-81.889965,"[Gas station, ATM, Convenience store, Restaurant]",1.8,Florida
159,Baby Food Grill & Bar,0x88d9ab98b5baad79:0x1a2e3f0642a55246,26.010250,-80.148659,[Mexican restaurant],3.0,Florida
...,...,...,...,...,...,...,...
703868,Park View Tavern,0x89c3b2b8e9c8c357:0xb1b07826ef85a598,40.653743,-74.234148,"[Bar, Italian restaurant, Pub, Restaurant]",4.3,New Jersey
703873,The Brown Derby Cafe & Rental Hall,0x89c24d60acc3ffff:0xc8048546518281f6,40.661250,-74.193971,"[Restaurant, Family restaurant]",4.6,New Jersey
703877,George's Lunch,0x89c24d7a198a3541:0x3dcb8f429c053248,40.654865,-74.177221,[American restaurant],4.5,New Jersey
703880,Las Brasas Sports Bar and Restaurant,0x89c3b290dcc12d6d:0x34ad59938efed577,40.661939,-74.248363,"[Latin American restaurant, Mexican restaurant...",3.9,New Jersey


In [35]:
nuevo_dataset['category'] = nuevo_dataset['category'].apply(lambda x: [item.lower() for item in x])

nuevo_dataset

C:\Users\jonat\AppData\Local\Temp\ipykernel_11380\3646945747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuevo_dataset['category'] = nuevo_dataset['category'].apply(lambda x: [item.lower() for item in x])


,name,gmap_id,latitude,longitude,category,stars,state
5,Cape Seafood Shack,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,26.641377,-81.940545,[restaurant],5.0,Florida
6,Fresh Point Country Buffet,0x88c2e4e34f1ed783:0x76c5da381c499d79,27.867489,-82.702971,[buffet restaurant],5.0,Florida
48,Hot Box,0x8890b9241e704667:0x3a1e565c17c00993,30.391411,-87.267220,[restaurant],4.2,Florida
83,Shell,0x88e635378f43352f:0xa1b53c63436fa428,29.183272,-81.889965,"[gas station, atm, convenience store, restaurant]",1.8,Florida
159,Baby Food Grill & Bar,0x88d9ab98b5baad79:0x1a2e3f0642a55246,26.010250,-80.148659,[mexican restaurant],3.0,Florida
...,...,...,...,...,...,...,...
703868,Park View Tavern,0x89c3b2b8e9c8c357:0xb1b07826ef85a598,40.653743,-74.234148,"[bar, italian restaurant, pub, restaurant]",4.3,New Jersey
703873,The Brown Derby Cafe & Rental Hall,0x89c24d60acc3ffff:0xc8048546518281f6,40.661250,-74.193971,"[restaurant, family restaurant]",4.6,New Jersey
703877,George's Lunch,0x89c24d7a198a3541:0x3dcb8f429c053248,40.654865,-74.177221,[american restaurant],4.5,New Jersey
703880,Las Brasas Sports Bar and Restaurant,0x89c3b290dcc12d6d:0x34ad59938efed577,40.661939,-74.248363,"[latin american restaurant, mexican restaurant...",3.9,New Jersey


In [36]:
empresas_concatenadas_restaurantes_estados_google = nuevo_dataset.copy()

Normalizamos los valores de categorias en formato lista.

In [ ]:
#Normalizamos los nombres de las columnas y actualizo index
empresas_concatenadas_restaurantes_estados_google = empresas_concatenadas_restaurantes_estados_google[['gmap_id', 'name', 'latitude', 'longitude', 'stars', 'category', 'state']]
empresas_concatenadas_restaurantes_estados_google.columns = ['gmap_id', 'name', 'latitude', 'longitude', 'stars', 'categories', 'state']
empresas_concatenadas_restaurantes_estados_google.reset_index(drop=True, inplace=True)

In [41]:
empresas_concatenadas_restaurantes_estados_google.to_parquet(r'..\datasets\processed\google\business_google.parquet.gz', compression='gzip')

In [4]:
empresas_concatenadas_restaurantes_estados_google = pd.read_parquet(r'..\datasets\processed\google\business_google.parquet.gz')
empresas_concatenadas_restaurantes_estados_google

,gmap_id,name,latitude,longitude,stars,categories,state
0,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Cape Seafood Shack,26.641377,-81.940545,5.0,[restaurant],Florida
1,0x88c2e4e34f1ed783:0x76c5da381c499d79,Fresh Point Country Buffet,27.867489,-82.702971,5.0,[buffet restaurant],Florida
2,0x8890b9241e704667:0x3a1e565c17c00993,Hot Box,30.391411,-87.267220,4.2,[restaurant],Florida
3,0x88e635378f43352f:0xa1b53c63436fa428,Shell,29.183272,-81.889965,1.8,"[gas station, atm, convenience store, restaurant]",Florida
4,0x88d9ab98b5baad79:0x1a2e3f0642a55246,Baby Food Grill & Bar,26.010250,-80.148659,3.0,[mexican restaurant],Florida
...,...,...,...,...,...,...,...
55174,0x89c3b2b8e9c8c357:0xb1b07826ef85a598,Park View Tavern,40.653743,-74.234148,4.3,"[bar, italian restaurant, pub, restaurant]",New Jersey
55175,0x89c24d60acc3ffff:0xc8048546518281f6,The Brown Derby Cafe & Rental Hall,40.661250,-74.193971,4.6,"[restaurant, family restaurant]",New Jersey
55176,0x89c24d7a198a3541:0x3dcb8f429c053248,George's Lunch,40.654865,-74.177221,4.5,[american restaurant],New Jersey
55177,0x89c3b290dcc12d6d:0x34ad59938efed577,Las Brasas Sports Bar and Restaurant,40.661939,-74.248363,3.9,"[latin american restaurant, mexican restaurant...",New Jersey
